# CUDA Lesson 1: Simple matrix multiplication

Author of the original: [Jeremy Howard](https://jeremy.fast.ai/), [Original notebook](https://colab.research.google.com/drive/180uk6frvMBeT4tywhhYXmz3PJaCIA_uk?usp=sharing&authuser=1), [Video](https://www.youtube.com/watch?v=4sgKnKbR-WE)


In the first part of his lecture Jeremy writes a kernel for RGB-to-greyscale conversion. I will skip that and start from the matmul kernel.

To slightly modify the algorithm I will instead implement SGEMM ($\textbf{C} = \alpha \cdot \textbf{AB} + \beta \textbf{C}$) with all matrices being square.


In [45]:
import torch
from torch import tensor
import torchvision as tv
import torchvision.transforms.functional as tvf
from torchvision import io
from torch.utils.cpp_extension import load_inline

import typing as t

T = torch.Tensor

In [46]:
# Creating dummy data

torch.manual_seed(1234)

n_dim = 200
A = torch.randn(n_dim, n_dim)
B = torch.randn(n_dim, n_dim)
C = torch.randn(n_dim, n_dim)

alpha = 3
beta = 1.5

Following Jeremy I will first write the SGEMM algorithm in Python

In [47]:
def sgemm(matrix_a: T, matrix_b: T, matrix_c: T, alpha: float, beta: float):
    out = beta * matrix_c.clone()
    dim = matrix_a.shape[0]

    for ii in range(dim):
        for jj in range(dim):
            for kk in range(dim):
                out[ii, jj] += alpha * (matrix_a[ii, kk] * matrix_b[kk, jj])
    return out

In [48]:
from types import SimpleNamespace as ns
import math

def block_kernel(f, blocks, threads, *args):
    for b0 in range(blocks.x):
        for b1 in range(blocks.y):
            for t0 in range(threads.x):
                for t1 in range(threads.y):
                    f(ns(x=b1, y=b0), ns(x=t1, y=t0), threads, *args)

def sgemm_single_thread(
    blockidx,
    threadidx,
    blockdim,
    matrix_a: T,
    matrix_b: T,
    matrix_c: T,
    alpha: float,
    beta: float,
    dim: int,
    ):
    col = blockidx.x * blockdim.x + threadidx.x
    row = blockidx.y * blockdim.y + threadidx.y

    if col >= dim or row >= dim:
        return
    tmp = 0
    for kk in range(dim):
        tmp += matrix_a[row*dim + kk] * matrix_b[col + kk*dim]
    matrix_c[row*dim + col] = tmp * alpha + beta * matrix_c[row*dim + col]


def sgemm_block(matrix_a: T, matrix_b: T, matrix_c: T, alpha: float, beta: float):
    dim = matrix_a.shape[0]
    tpb = ns(x=16, y=16)
    blocks = ns(
        x=math.ceil(dim/tpb.x),
        y=math.ceil(dim/tpb.y),
    )

    block_kernel(
        sgemm_single_thread,
        blocks,
        tpb,
        matrix_a.flatten(),
        matrix_b.flatten(),
        matrix_c.flatten(),
        alpha,
        beta,
        dim,
    )
    return matrix_c.reshape(dim, dim)

In [49]:
pt_out = alpha * torch.mm(A, B) + beta * C

In [51]:
# sgemm_out=sgemm(A, B, C, alpha, beta)
# assert torch.isclose(sgemm_out, pt_out, atol=1e-5).all()

KeyboardInterrupt: 

In [ ]:
# sgemm_block_out=sgemm_block(A, B, C, alpha, beta)
# assert torch.isclose(sgemm_block_out, pt_out, atol=1e-5).all()

# CUDA setup


> `CUDA_LAUNCH_BLOCKING=1` is a debug env variable used to block kernel launches and to report the proper stacktrace once an assert is triggered. You should not use it in production, but only during debugging.


[source](https://discuss.pytorch.org/t/cuda-launch-blocking-1-reduces-the-training-speed/160924)

In [52]:
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [ ]:
# wurlitzer
# Capture C-level stdout/stderr pipes in Python
# More here: https://eli.thegreenplace.net/2015/redirecting-all-kinds-of-stdout-in-python/

# ninja for build
%pip install -q wurlitzer ninja

In [ ]:
%load_ext wurlitzer

In [53]:
def load_cuda(cuda_src, cpp_src, funcs, opt=False, verbose=False):
    return load_inline(
        cuda_sources=[cuda_src],
        cpp_sources=[cpp_src],
        functions=funcs,
        extra_cuda_cflags=["-O2"] if opt else [],
        verbose=verbose,
        name="inline_ext",
    )

In [54]:
# Utility functions defined by Jeremy

cuda_begin = r'''
#include <torch/extension.h>
#include <stdio.h>
#include <c10/cuda/CUDAException.h>

#define CHECK_CUDA(x) TORCH_CHECK(x.device().is_cuda(), #x " must be a CUDA tensor");
#define CHECK_CONTIGUOUS(x) TORCH_CHECK(x.is_contiguous(), #x " must be contiguous");
#define CHECK_INPUT(x) CHECK_CUDA(x); CHECK_CONTIGUOUS(x);

inline unsigned int cdiv(unsigned int a, unsigned int b) { return (a + b - 1) / b;}
'''

In [55]:
cuda_src = cuda_begin + r'''
__global__ void sgemm_kernel(float* matrix_a, float* matrix_b, float* matrix_c, float alpha, float beta, int dim) {
    int row = blockIdx.y*blockDim.y + threadIdx.y;
    int col = blockIdx.x*blockDim.x + threadIdx.x;

    if (row >= dim || col >= dim) return;
    float tmp = 0;
    for (int i = 0; i<dim; ++i) {
        tmp += (matrix_a[row*dim + i] * matrix_b[dim*i + col]);
    }
    matrix_c[row*dim + col] = alpha * tmp + beta * matrix_c[row*dim + col];
}

torch::Tensor sgemm(torch::Tensor matrix_a, torch::Tensor matrix_b, torch::Tensor matrix_c, float alpha, float beta) {
    CHECK_INPUT(matrix_a); CHECK_INPUT(matrix_b); CHECK_INPUT(matrix_c);
    int dim = matrix_a.size(0);

    dim3 tpb(16,16);
    dim3 blocks(cdiv(dim, tpb.x), cdiv(dim, tpb.y));

    sgemm_kernel<<<blocks, tpb>>>(
        matrix_a.data_ptr<float>(),
        matrix_b.data_ptr<float>(),
        matrix_c.data_ptr<float>(),
        alpha,
        beta,
        dim
        );
    C10_CUDA_KERNEL_LAUNCH_CHECK();
    return matrix_c;
}
'''

In [56]:
cpp_src = "torch::Tensor sgemm(torch::Tensor matrix_a, torch::Tensor matrix_b, torch::Tensor matrix_c, float alpha, float beta);"

In [57]:
module = load_cuda(cuda_src, cpp_src, ['sgemm'], verbose=True)

Using /root/.cache/torch_extensions/py311_cu124 as PyTorch extensions root...
The input conditions for extension module inline_ext have changed. Bumping to version 1 and re-building as inline_ext_v1...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py311_cu124/inline_ext/build.ninja...
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module inline_ext_v1...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] c++ -MMD -MF main.o.d -DTORCH_EXTENSION_NAME=inline_ext_v1 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /usr/local/lib/python3.11/dist-packages/torch/include -isystem /usr/local/lib/python3.11/dist-packages/torch/include/torch/csrc/api/include -isystem /usr/local/lib/python3.11/dist-packages/torch/include/TH -isystem /usr/local/lib/python3.11/dist-packages/torch/include/THC -isystem /usr/local/cuda/include -isystem /usr/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -c /root/.cache/torch_extensions/py311_cu124/inline_ext/main.cpp -o main.o 
[2/3] /usr/local/cuda/bin/nvcc --generate-dependencies-with-compile --dependency-output cuda.cuda.o.d -DTORCH_EXTENSION_NAME=inline_ext_v1 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /usr/local/lib/python3.11/dist-packages/torch/i

Loading extension module inline_ext_v1...


In [58]:
a_gpu = A.contiguous().cuda()
b_gpu = B.contiguous().cuda()
c_gpu = C.contiguous().cuda()

In [59]:
%time sgemm_cuda_output = module.sgemm(a_gpu, b_gpu, c_gpu, alpha, beta)

CPU times: user 7.66 ms, sys: 7 µs, total: 7.67 ms
Wall time: 7.65 ms


In [73]:
diff = sgemm_cuda_output.cpu() - pt_out

In [74]:
diff.max()

tensor(0.0006)

In [75]:
print(cuda_src)


#include <torch/extension.h>
#include <stdio.h>
#include <c10/cuda/CUDAException.h>

#define CHECK_CUDA(x) TORCH_CHECK(x.device().is_cuda(), #x " must be a CUDA tensor");
#define CHECK_CONTIGUOUS(x) TORCH_CHECK(x.is_contiguous(), #x " must be contiguous");
#define CHECK_INPUT(x) CHECK_CUDA(x); CHECK_CONTIGUOUS(x);

inline unsigned int cdiv(unsigned int a, unsigned int b) { return (a + b - 1) / b;}

__global__ void sgemm_kernel(float* matrix_a, float* matrix_b, float* matrix_c, float alpha, float beta, int dim) {
    int row = blockIdx.y*blockDim.y + threadIdx.y;
    int col = blockIdx.x*blockDim.x + threadIdx.x;

    if (row >= dim || col >= dim) return;
    float tmp = 0;
    for (int i = 0; i<dim; ++i) {
        tmp += (matrix_a[row*dim + i] * matrix_b[dim*i + col]);
    }
    matrix_c[row*dim + col] = alpha * tmp + beta * matrix_c[row*dim + col];
}

torch::Tensor sgemm(torch::Tensor matrix_a, torch::Tensor matrix_b, torch::Tensor matrix_c, float alpha, float beta) {
    CHECK_INPUT

In [77]:
print(cpp_src)

torch::Tensor sgemm(torch::Tensor matrix_a, torch::Tensor matrix_b, torch::Tensor matrix_c, float alpha, float beta);
